In [ ]:
%pip install beacon-api --upgrade

In [ ]:
from beacon_api import *
import numpy as np
import os
BLUE_CLOUD_JWT_TOKEN = os.getenv("BLUE_CLOUD_JWT_TOKEN") 
client = Client("https://beacon-argo.maris.nl", jwt_token=BLUE_CLOUD_JWT_TOKEN)

In [ ]:
tables = client.list_tables()
tables

In [ ]:
tables['default'].get_table_schema()

In [ ]:
# METADATA
from datetime import datetime, timezone
dt_str = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

query_builder = tables['default'].query()

query_builder = (
    query_builder
    .add_select_column("__entry_key")
    
    .add_select(SelectLiteral("BEACON_ARGO", alias="SOURCE_BDI"))
    .add_select(SelectLiteral("2026-01-14T00:00:00Z", "COMMON_BDI_SNAPSHOT_DATE"))
    .add_select(SelectLiteral("1.5.0", "COMMON_BDI_MONOLITH_VERSION"))
    .add_select(SelectLiteral(dt_str, "COMMON_HARMONIZATION_DATE"))
    
    .add_select(SelectFunction("concat", [SelectColumn("PLATFORM_NUMBER"), SelectLiteral("_"), SelectColumn("CYCLE_NUMBER"), SelectLiteral("_"), SelectColumn("DIRECTION")], alias="SOURCE_BDI_DATASET_ID"))
    .add_select(SelectColumn("PLATFORM_NUMBER"))
    .add_select(SelectColumn("DIRECTION"))
    .add_select(SelectColumn("CYCLE_NUMBER"))
    .add_select(SelectColumn("WMO_INST_TYPE"))
    .add_select(SelectColumn(".featureType"))
    .add_select(SelectColumn(".featureType", alias="COMMON_FEATURE_TYPE"))
    .add_select(SelectFunction("concat", [SelectColumn("__entry_key"), SelectColumn("CYCLE_NUMBER"), SelectColumn("DIRECTION")], "COMMON_ODV_TAG"))
    .add_select(SelectFunction("map_argo_platform_l06", [Functions.try_cast_to_type(SelectLiteral(0), np.int64, "platform_l06")], alias="COMMON_PLATFORM_L06"))
    .add_select(SelectFunction("map_argo_platform_edmo", [SelectFunction("btrim", [SelectColumn("PLATFORM_NUMBER")])], alias="COMMON_EDMO_CODE"))
)

In [ ]:
# TIME, LOCATION, DEPTH
query_builder = (
    query_builder
    .add_select(SelectColumn("JULD"))
    .add_select(SelectColumn("JULD_QC"))
    .add_select(SelectColumn("JULD", alias="COMMON_TIME"))
    .add_select(SelectColumn("JULD_QC", alias="COMMON_TIME_QC"))
    
    .add_select(SelectColumn("LATITUDE"))
    .add_select(SelectColumn("LATITUDE.standard_name"))
    .add_select(SelectColumn("LATITUDE.units"))
    .add_select(SelectColumn("LATITUDE", alias="COMMON_LATITUDE"))
    .add_select(SelectLiteral("degrees_north", "COMMON_LATITUDE_UNITS"))
    .add_select(SelectLiteral("Latitude", "COMMON_LATITUDE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ALATZZ01", "COMMON_LATITUDE_P01"))
    .add_select(SelectLiteral("SDN:P06::DEGN", "COMMON_LATITUDE_P06"))
    
    .add_select(SelectColumn("LONGITUDE"))
    .add_select(SelectColumn("LONGITUDE.standard_name"))
    .add_select(SelectColumn("LONGITUDE.units"))
    .add_select(SelectColumn("LONGITUDE", alias="COMMON_LONGITUDE"))
    .add_select(SelectLiteral("degrees_east", "COMMON_LONGITUDE_UNITS"))
    .add_select(SelectLiteral("Longitude", "COMMON_LONGITUDE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ALONZZ01", "COMMON_LONGITUDE_P01"))
    .add_select(SelectLiteral("SDN:P06::DEGE", "COMMON_LONGITUDE_P06"))
    
    .add_select(SelectColumn("POSITION_QC"))
    .add_select(SelectColumn("POSITION_QC", alias="COMMON_POSITION_QC"))
    
    .add_select_column("PRES")
    .add_select_column("PRES_ADJUSTED")
    .add_select_column("PRES.units")
    .add_select_column("PRES_ADJUSTED.units")
    .add_select_column("PRES_QC")
    .add_select_column("PRES_ADJUSTED_QC")
    .add_select_coalesced(["PRES_ADJUSTED", "PRES"], "COMMON_ORIGIN_DEPTH")
    .add_select_coalesced(["PRES_ADJUSTED_QC", "PRES_QC"], "COMMON_ORIGIN_DEPTH_QC")
    
    .add_select(SelectColumn("PRES.units", "COMMON_ORIGIN_DEPTH_UNITS"))
    .add_select(SelectLiteral("PRES.standard_name", "COMMON_ORIGIN_DEPTH_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::PRESPR01", "COMMON_ORIGIN_DEPTH_P01"))
    .add_select(SelectLiteral("SDN:P06::UPDB", "COMMON_ORIGIN_DEPTH_P06"))

    .add_select(SelectFunction("pressure_to_depth_teos_10", [SelectFunction("coalesce", [SelectColumn("PRES_ADJUSTED"), SelectColumn("PRES")]), SelectColumn("LATITUDE")], "COMMON_DEPTH"))
    .add_select(SelectFunction("coalesce", [SelectColumn("PRES_ADJUSTED_QC"), SelectColumn("PRES_QC")], "COMMON_DEPTH_QC"))
    .add_select(SelectLiteral("m", "COMMON_DEPTH_UNITS"))
    .add_select(SelectLiteral("depth", "COMMON_DEPTH_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ADEPZZ01", "COMMON_DEPTH_P01"))
    .add_select(SelectLiteral("SDN:P06::ULAA", "COMMON_DEPTH_P06"))
)

In [ ]:
# TEMPERATURE
query_builder = (
    query_builder
    .add_select(SelectColumn("TEMP"))
    .add_select(SelectColumn("TEMP_QC"))
    .add_select(SelectColumn("TEMP_ADJUSTED"))
    .add_select(SelectColumn("TEMP_ADJUSTED_QC"))
    .add_select_coalesced(["TEMP_ADJUSTED", "TEMP"], "COMMON_TEMPERATURE")
    .add_select_coalesced(["TEMP_ADJUSTED_QC", "TEMP_QC"], "COMMON_TEMPERATURE_QC")
    .add_select(SelectLiteral("SDN:P01::TEMPPR01", "COMMON_TEMPERATURE_P01"))
    .add_select(SelectLiteral("SDN:P06::UPAA", "COMMON_TEMPERATURE_P06"))
    .add_select(SelectLiteral("sea_water_temperature", "COMMON_TEMPERATURE_STANDARD_NAME"))
    .add_select(SelectLiteral("degree_C", "COMMON_TEMPERATURE_UNITS"))
    .add_select(SelectFunction("map_argo_instrument_l05", [Functions.try_cast_to_type(SelectLiteral(0), np.int64, "temp_l05")], "COMMON_TEMPERATURE_L05"))
    .add_select(SelectFunction("map_wmo_instrument_type_l22", [SelectFunction("btrim",[SelectColumn("WMO_INST_TYPE")])], "COMMON_TEMPERATURE_L22"))
)

In [ ]:
# Salinity
query_builder = (
    query_builder
    .add_select(SelectColumn("PSAL"))
    .add_select(SelectColumn("PSAL_QC"))
    .add_select(SelectColumn("PSAL_ADJUSTED"))
    .add_select(SelectColumn("PSAL_ADJUSTED_QC"))
    .add_select_coalesced(["PSAL_ADJUSTED", "PSAL"], "COMMON_SALINITY")
    .add_select_coalesced(["PSAL_ADJUSTED_QC", "PSAL_QC"], "COMMON_SALINITY_QC")
    .add_select(SelectLiteral("SDN:P01::PSLTZZ01", "COMMON_SALINITY_P01"))
    .add_select(SelectLiteral("SDN:P06::UUUU", "COMMON_SALINITY_P06"))
    .add_select(SelectLiteral("sea_water_practical_salinity", "COMMON_SALINITY_STANDARD_NAME"))
    .add_select(SelectLiteral("dimensionless", "COMMON_SALINITY_UNITS"))
    .add_select(SelectFunction("map_argo_instrument_l05", [Functions.try_cast_to_type(SelectLiteral(0), np.int64, "psal_l05")], "COMMON_SALINITY_L05"))
    .add_select(SelectFunction("map_wmo_instrument_type_l22", [SelectFunction("btrim",[SelectColumn("WMO_INST_TYPE")])], "COMMON_SALINITY_L22"))
)

In [ ]:
import os 
import json

# Make output dir
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)
# for each year between 1950 and 2025
for year in range(1950, 2026):
    print(f"Processing year {year}")
    query_builder.filters = []
    query_builder = query_builder.add_range_filter("COMMON_TIME", f"{year}-01-01T00:00:00Z", f"{year}-12-31T23:59:59Z")
    query_builder = (
        query_builder.add_filter(OrFilter([
            IsNotNullFilter("TEMP"),
            IsNotNullFilter("PSAL"),
        ]))
    )
    
    output_file = os.path.join(output_dir, f"argo_{year}.parquet")
    try:
        query_builder.to_parquet(output_file)
        print(f"Saved to {output_file}")
    except Exception as e:
        print(f"Failed to process year {year}: {e}")